[HOME](./__README__.ipynb) | [Login](./Login.ipynb) | [Database](./Database.ipynb) | [Cleanup Runbook](./Cleanup.ipynb)

# Preset Runbook

In [2]:
# Import required library and create log folder 
import ipylivebash
import ipywidgets as widgets
from IPython.display import display
import shutil
!mkdir -p logs
!echo Completed

Completed


# Choose Environment

Pick from "dev", "uat" and "prod".

*Remarks:* It just simulate a real project but it is meaningless for a local network

In [ ]:
def run():
    print("choose an environment")
    def confirm(environment):
        shutil.copy(f"../django-cms-quickstart/secrets/config.env-{environment}",  "../django-cms-quickstart/config.env")
        with output_widget:
            print(f"Confirmed environment: {environment}. File 'config.env' updated.")

    environment_select = widgets.Select(
        options=["dev", "uat", "prod"],
        disabled=False
    )
    confirm_button = widgets.Button(description="Confirm")
    output_widget = widgets.Output(layout={'width': '95%'})

    def on_confirm_button_clicked(b):
        confirm(environment_select.value)

    confirm_button.on_click(on_confirm_button_clicked)
    display(widgets.VBox([environment_select, confirm_button, output_widget]))
run()

# Setup

In [ ]:
%%livebash --save logs/setup.log --notify --save-timestamp

cd ../django-cms-quickstart
docker compose up -d database_default
docker compose run --rm web python manage.py migrate


# Create Superuser

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def run():
    print("Enter password")
    def confirm(password):
        with open('admin.pass', 'w') as file:
            file.write(password)
        output_widget.append_stdout(f"Password saved to admin.pass\n")

    password_input = widgets.Password(layout={'width': '50%'})

    confirm_button = widgets.Button(description='Confirm')
    confirm_button.on_click(lambda _: confirm(password_input.value))
    output_widget = widgets.Output(layout={'width': '95%', 'border': '1px solid black'})

    widgets_box = widgets.VBox([password_input, confirm_button, output_widget])
    display(widgets_box)
run()

In [ ]:
%%livebash --ask-confirm
# Create superuser

ADMIN_PASSWD=`cat admin.pass`

CMD=$(cat << EOF
export DJANGO_SUPERUSER_PASSWORD="${ADMIN_PASSWD}"
python manage.py createsuperuser --noinput --username admin --email tester@example.com
EOF
)

cd ../django-cms-quickstart
docker compose up -d database_default
docker compose run web bash -c "$CMD"


# Startup

In [ ]:
%%livebash --height 5

cd ../django-cms-quickstart
docker-compose up -d

In [ ]:
%%livebash --height 8
# Check running services

cd ../django-cms-quickstart
docker-compose ps

* * *

Now it is ready, open this [link](http://localhost:8080/admin) to visit the running service

[Back to top](#Preset-Runbook)